In [1]:
import pandas as pd
import numpy as np

df =pd.read_csv("data.csv",index_col=0)

#Q1
column_mapping= {
    'SC155Q01HA': 'DeviceCount',
    'SC155Q02HA': 'Bandwidth',
    'SC155Q03HA': 'InstructionDevices',
    'SC155Q04HA': 'ComputingPower',
    'SC155Q05HA': 'Software'
}
df.rename(columns=column_mapping,inplace=True)
#inplace=True表示直接在原数据上修改
df=df[list(column_mapping.values())].dropna()
#删除缺失值所在的行

simplified_names = ['DeviceCount', 'Bandwidth', 'InstructionDevices', 'ComputingPower', 'Software']


In [2]:
#Q2
#构建索引
ind2val = {}
val2ind={}
index = 0
simplified_names = ['DeviceCount', 'Bandwidth', 'InstructionDevices', 'ComputingPower', 'Software']
int_size=0

for column in simplified_names:
    unique_values = df[column].unique()
    for value in unique_values:
        ind2val[index] = f"{column}={value}"
        val2ind[f"{column}={value}"] = index
        index += 1
        int_size+=1

#替换特征值为索引
df=df.apply(lambda x:x.map(lambda v:val2ind[f"{x.name}={v}"]))
# print(df['Software'])


In [4]:
#T2
def count(dt, item):#计算item在dt中的出现次数
    cnt = 0
    for line in dt:
        if set(item) & set(line) == set(item):
            cnt += 1
    return cnt

def apriori(df,min_support,int_size):
    dt=df.to_numpy()
    N=len(dt)
    frequent_itemsets={1:[]}
    for i in range(int_size+1):
        cnt=count(dt,[i])
        if cnt>=min_support*N:
            frequent_itemsets[1].append(([i],cnt))
    #求每个数字的出现次数并筛选，即1阶频繁项集的候选集
    k=2
    while True:
        candidates=[]
        frequent_new=[]
        #此处如果不用frequent_new,而直接用frequent_itemsets[k]，则会结果中会出现空集
        for i in range(len(frequent_itemsets[k-1])-1):
        #len(frequent_itemsets[k-2])表示1，2，3.。。阶频繁项集的长度
        #-1以防止超出索引
            for j in range(i+1,len(frequent_itemsets[k-1])):
                if frequent_itemsets[k-1][i][0][:-1]==frequent_itemsets[k-1][j][0][:-1]:
                    candidates.append(sorted(set(frequent_itemsets[k-1][i][0]) | set(frequent_itemsets[k-1][j][0])))
                    #set()去重，|求并集，sorted()排序
                    #得到k阶频繁项集的候选集
        # candidates=pd.DataFrame(candidates)
        for candidate in candidates:
            cnt=count(dt,candidate)
            #candidates.iloc[i,:]表示第i行的所有元素，是一个列表
            if cnt>=min_support*N:
                frequent_new.append((candidate,cnt))
                #将k阶频繁项集的候选集加入到k阶频繁项集中
        if len(frequent_new)==0:
            break
        else:
            frequent_itemsets[k]=frequent_new
        k+=1
    return frequent_itemsets


fre_half=apriori(df,0.5,int_size)
fre_quarter=apriori(df,0.25,int_size)

print(fre_half)
print(fre_quarter)

{1: [([1], 12930), ([3], 12873), ([5], 11323), ([7], 12341), ([9], 13080)], 2: [([1, 3], 10637), ([1, 5], 10506), ([1, 7], 10357), ([1, 9], 10596), ([7, 9], 10705)]}
{1: [([0], 7751), ([1], 12930), ([2], 7808), ([3], 12873), ([4], 9358), ([5], 11323), ([6], 8340), ([7], 12341), ([8], 7601), ([9], 13080)], 2: [([0, 2], 5515), ([0, 4], 6934), ([0, 6], 5767), ([0, 8], 5267), ([1, 3], 10637), ([1, 5], 10506), ([1, 7], 10357), ([1, 9], 10596), ([2, 4], 5856), ([2, 6], 5625), ([3, 5], 9371), ([3, 7], 10158), ([3, 9], 10223), ([4, 6], 6813), ([4, 8], 6068), ([5, 7], 9796), ([5, 9], 9790), ([6, 8], 5965), ([7, 9], 10705)], 3: [([0, 4, 6], 5452), ([1, 3, 5], 9079), ([1, 3, 7], 9142), ([1, 3, 9], 9139), ([1, 5, 7], 9294), ([1, 5, 9], 9306), ([1, 7, 9], 9387), ([3, 5, 7], 8563), ([3, 5, 9], 8445), ([3, 7, 9], 9099), ([4, 6, 8], 5253), ([5, 7, 9], 8975)], 4: [([1, 3, 5, 7], 8362), ([1, 3, 5, 9], 8245), ([1, 3, 7, 9], 8404), ([1, 5, 7, 9], 8622), ([3, 5, 7, 9], 7984)], 5: [([1, 3, 5, 7, 9], 7834)]}

In [ ]:
#重新构建索引
ind2val = {}
val2ind={}
index = 0
int_size=0

#先将原始取值简化
for column in simplified_names:
    unique_values = sorted(df[column].unique())
    for value in unique_values:

        if value==1 or value==2:
            df[column]=df[column].replace(value,0)
        elif value==3 or value==4:
            df[column]=df[column].replace(value,1)
        #将1和2替换为0，3和4替换为1

for column in simplified_names:
    unique_values = df[column].unique()
    for value in unique_values:
        ind2val[index] = f"{column}={value}"
        val2ind[f"{column}={value}"] = index
        index += 1
        int_size+=1

#替换特征值为索引
df=df.apply(lambda x:x.map(lambda v:val2ind[f"{x.name}={v}"]))


{'DeviceCount=0.0': 0, 'DeviceCount=1.0': 1, 'Bandwidth=0.0': 2, 'Bandwidth=1.0': 3, 'InstructionDevices=0.0': 4, 'InstructionDevices=1.0': 5, 'ComputingPower=0.0': 6, 'ComputingPower=1.0': 7, 'Software=0.0': 8, 'Software=1.0': 9}


In [ ]:
def get_support_count(freq_dict, itemset):
    k = len(itemset)
    if k not in freq_dict:
        return 0
    sorted_itemset = sorted(itemset)
    for (items, cnt) in freq_dict[k]:
        if items == sorted_itemset:
            return cnt
    return 0

def extract_rules(freq_itemsets, Y, min_conf, N):
    Y = [1]
    rules = []
    # 获取Y的支持数
    y_support = get_support_count(freq_itemsets, Y)
    if y_support == 0:
        return rules
  
    # 遍历所有k≥2的频繁项集
    for k in list(freq_itemsets.keys()):
        if k < 2:
            continue
        for (itemset, support_XY) in freq_itemsets[k]:
            if 1 in itemset:
                X = sorted([item for item in itemset if item != 1])
                if not X:
                    continue
                # 检查X是否为频繁项集
                x_support = get_support_count(freq_itemsets, X)
                if x_support == 0:
                    continue
                # 计算置信度
                confidence = support_XY / x_support
                if confidence >= min_conf:
                    # 计算提升度
                    lift = (support_XY * N) / (x_support * y_support)
                    rules.append((X, confidence, lift))
    return rules

dt=df.to_numpy()
# 假设N为数据集总行数
N = len(dt)  # dt是df转换后的numpy数组

# 从支持度0.25的频繁项集中提取规则
rules = extract_rules(fre_half, Y=[1], min_conf=0.8, N=N)

# 输出结果
print("关联规则 X -> {1}（置信度≥0.8）:")
for (X, conf, lift) in rules:
    print(f"规则 {X} → {{1}}: 置信度={conf:.2f}, 提升度={lift:.2f}")




关联规则 X -> {1}（置信度≥0.8）:
规则 [3] → {1}: 置信度=0.83, 提升度=1.32
规则 [5] → {1}: 置信度=0.93, 提升度=1.48
规则 [7] → {1}: 置信度=0.84, 提升度=1.34
规则 [9] → {1}: 置信度=0.81, 提升度=1.30


In [5]:
def get_support_count(freq_dict, itemset):
    k = len(itemset)
    if k not in freq_dict:
        return 0
    sorted_itemset = sorted(itemset)
    for (items, cnt) in freq_dict[k]:
        if items == sorted_itemset:
            return cnt
    return 0

def extract_rules(freq_itemsets, Y, min_conf, N):
    Y = [1]
    rules = []
    # 获取Y的支持数
    y_support = get_support_count(freq_itemsets, Y)
    if y_support == 0:
        return rules
  
    # 遍历所有k≥2的频繁项集
    for k in list(freq_itemsets.keys()):
        if k < 2:
            continue
        for (itemset, support_XY) in freq_itemsets[k]:
            if 1 in itemset:
                X = sorted([item for item in itemset if item != 1])
                if not X:
                    continue
                # 检查X是否为频繁项集
                x_support = get_support_count(freq_itemsets, X)
                if x_support == 0:
                    continue
                # 计算置信度
                confidence = support_XY / x_support
                if confidence >= min_conf:
                    # 计算提升度
                    lift = (support_XY * N) / (x_support * y_support)
                    rules.append((X, confidence, lift))
    return rules

dt=df.to_numpy()
# 假设N为数据集总行数
N = len(dt)  # dt是df转换后的numpy数组

# 从支持度0.25的频繁项集中提取规则
rules = extract_rules(fre_half, Y=[1], min_conf=0.8, N=N)

# 输出结果
print("关联规则 X -> {1}（置信度≥0.8）:")
for (X, conf, lift) in rules:
    print(f"规则 {X} → {{1}}: 置信度={conf:.2f}, 提升度={lift:.2f}")




关联规则 X -> {1}（置信度≥0.8）:
规则 [3] → {1}: 置信度=0.83, 提升度=1.32
规则 [5] → {1}: 置信度=0.93, 提升度=1.48
规则 [7] → {1}: 置信度=0.84, 提升度=1.34
规则 [9] → {1}: 置信度=0.81, 提升度=1.30


范围都是1-4
SC155Q01HA
The number of digital devices 
connected to the Internet is sufficient

SC155Q02HA
The school’s Internet bandwidth or speed is sufficient 

SC155Q03HA
The number of digital devices for instruction is sufficient


SC155Q04HA
Digital devices at the school are sufficiently powerful in terms of computing capacity

SC155Q05HA
The availability of adequate software is sufficient 